In [1]:
import pandas as pd
import json

# Populate cards_database.json with external database

## Load cards_datatabase.json

In [2]:
def json_to_df_and_excel():
    # Load the JSON file
    with open(r"C:\Users\RubénLopez\Documents\mtg_decklists_analyzer\cards_database.json", "r") as file:
        cards_database = json.load(file)

    # Flatten the JSON structure
    data = []
    for format, cards in cards_database.items():
        for card_name, attributes in cards.items():
            row = {
                'format': format,
                'card_name': card_name,
                'type': attributes.get('type'),
                'subtype': attributes.get('subtype'),
                'color': attributes.get('color'),
            }
            data.append(row)

    # Convert the data into a DataFrame
    df = pd.DataFrame(data)

    # Export the DataFrame to an Excel file
    df.to_excel('cards_database.xlsx', index=False)

    return df

In [3]:
df_json = json_to_df_and_excel()
df_json

,format,card_name,type,subtype,color
0,Premodern,Mox Diamond,Acceleration,Tier 1,C
1,Premodern,Hydroblast,Hate,Red,U
2,Premodern,Blue Elemental Blast,Hate,Red,U
3,Premodern,Counterspell,Counter,Tier 1,U
4,Premodern,Mana Leak,Counter,Tier 1,U
...,...,...,...,...,...
680,Premodern,Massacre,Sorcery,unknown,B
681,Premodern,Mind Harness,Enchantment,unknown,U
682,Premodern,Crippling Fatigue,Sorcery,unknown,B
683,Premodern,Unsummon,Instant,unknown,U


In [4]:
df_json[df_json.card_name=="Attunement"]

,format,card_name,type,subtype,color
638,Premodern,Attunement,Enchantment,unknown,U


## Load external database

In [5]:
df_all_cards = pd.read_csv("output_modified.csv", index_col=0)
df_all_cards

,cardName,superType,cardType,subType,typeNum,manaCost,convertedMana,cardSet,rarity,rules,color
0,"""Ach! Hans, Run!""",NaN,Enchantment,NaN,NaN,2RRGG,6,Unhinged,Rare,"At the beginning of your upkeep, you may say ""...",RG
1,"""Rumors of My Death . . .""",NaN,Enchantment,NaN,NaN,2B,3,Unstable,Uncommon,"3B, Exile a permanent you control with a Leagu...",B
2,_____,NaN,Creature,Shapeshifter,1/1,1U,2,Unhinged,Uncommon,1: This card's name becomes the card name of y...,U
3,A Good Thing,NaN,Enchantment,NaN,NaN,4WB,6,Mystery Booster,Common,Spells and abilities you control can't destroy...,WB
4,Abandon Hope,NaN,Sorcery,NaN,NaN,X1B,2,Tempest,Uncommon,"As an additional cost to cast this spell, disc...",B
...,...,...,...,...,...,...,...,...,...,...,...
39040,Zurgo Helmsmasher,Legendary,Creature,Orc Warrior,7/2,2RWB,5,Duel Decks: Speed vs. Cunning,Mythic Rare,Haste\r\nZurgo Helmsmasher attacks each combat...,WBR
39042,Zur's Weirding,NaN,Enchantment,NaN,NaN,3U,4,Fifth Edition,Rare,Players play with their hands revealed.\r\nIf ...,U
39047,"Zurzoth, Chaos Rider",Legendary,Creature,Devil,2/3,2R,3,Jumpstart,Rare,Whenever an opponent draws their first card ea...,R
39048,"Zyym, Mesmeric Lord",Legendary,Creature,Vampire Wizard,3/4,1UBB,4,Mystery Booster,Common,"Flying, deathtouch\r\nWhen CARDNAME enters the...",UB


## Populate cards_database with external database

In [17]:
def populate_column(column_cards_db, column_ext, df_cards_db, df_ext):
    # Set the card names as the index
    df_cards_db.set_index('card_name', inplace=True)
    df_ext.set_index('cardName', inplace=True)

    # Replace 'Unknown' colors in df_json with colors from df_all_cards
    df_cards_db[column_cards_db] = df_cards_db[column_cards_db].where(df_cards_db[column_cards_db] != 'unknown', df_cards_db.index.map(df_ext[column_ext]))

    # Reset the index
    df_cards_db.reset_index(inplace=True)
    df_ext.reset_index(inplace=True)

    return df_cards_db

In [19]:
df_json = populate_column("color", "color", df_json, df_all_cards)
df_json = populate_column("type", "cardType", df_json, df_all_cards)

In [20]:
df_json = df_json[['format', 'card_name', 'type', 'subtype', 'color']]
df_json

,format,card_name,type,subtype,color
0,Premodern,Mox Diamond,Acceleration,Tier 1,C
1,Premodern,Hydroblast,Hate,Red,U
2,Premodern,Blue Elemental Blast,Hate,Red,U
3,Premodern,Counterspell,Counter,Tier 1,U
4,Premodern,Mana Leak,Counter,Tier 1,U
...,...,...,...,...,...
680,Premodern,Massacre,Sorcery,unknown,B
681,Premodern,Mind Harness,Enchantment,unknown,U
682,Premodern,Crippling Fatigue,Sorcery,unknown,B
683,Premodern,Unsummon,Instant,unknown,U


In [22]:
df_json[df_json["card_name"]=="Attunement"]

,format,card_name,type,subtype,color
638,Premodern,Attunement,Enchantment,unknown,Blue


# Update cards_database.json

In [22]:
def df_to_json(df_json):
    cards_database = {}
    for row in df_json.iterrows():
        row_list = row[1].values
        if row_list[0] not in cards_database:
            cards_database[row_list[0]] = {}
        if row_list[1] not in cards_database[row_list[0]]:
            cards_database[row_list[0]][row_list[1]] = {}
        cards_database[row_list[0]][row_list[1]]["type"] = row_list[2]
        cards_database[row_list[0]][row_list[1]]["subtype"] = row_list[3]
        cards_database[row_list[0]][row_list[1]]["color"] = row_list[4]
    # Save cards_database dict
    with open("cards_database.json", "w") as file:
        json.dump(cards_database, file)

In [23]:
df_to_json(df_json)

In [ ]:
# # Step 1: Define the function
# def process_mana_cost(mana_cost):
#     # Remove 'C'
#     mana_cost = mana_cost.replace('C', '')
#     # Eliminate duplicates
#     mana_cost = ''.join(sorted(set(mana_cost), key=mana_cost.index))
#     # Sort in the order 'WUBRG'
#     mana_cost = ''.join(sorted((c for c in mana_cost if c in "WUBRG"), key="WUBRG".index))
#     return mana_cost

# # Step 2: Apply the function
# df_all_cards.loc[df_all_cards['color'] == 'M', 'color'] = df_all_cards.loc[df_all_cards['color'] == 'M', 'manaCost'].apply(process_mana_cost)

In [ ]:
# colors_map = {
#     "White": "W",
#     "Blue": "U",
#     "Black": "B",
#     "Red": "R",
#     "Green": "G",
#     "Colorless": "C",
#     "Multicolored": "M",
# }
# df_all_cards["color"] = df_all_cards["color"].map(colors_map)

In [ ]:
# # Filter the DataFrame
# df_lands = df_all_cards[df_all_cards['cardType'] == 'Land'].copy()

# # Define the regular expression pattern
# pattern = r"(?<=\s)(W|U|B|R|G|C|any color)(?=\s|[.,])"

# # Use str.extractall to find all matches in the 'rules' column
# matches = df_lands['rules'].str.extractall(pattern)

# # The result is a MultiIndex DataFrame where the first level of the index
# # is the original index of df_lands and the second level is the match number
# # We can group by the first level of the index and join the matches with ','
# colors = matches.groupby(level=0)[0].apply(''.join)

# # Add the colors to df_lands as a new column
# df_lands['color'] = colors

# # Replace 'any color' with 'WUBRG'
# df_lands["color"] = df_lands["color"].str.replace("any color", "WUBRG")

# # Fill missing values with an empty string
# df_lands["color"] = df_lands["color"].fillna("")

# # Remove symbol duplicates
# df_lands['color'] = df_lands['color'].apply(lambda x: ''.join(sorted(set(x), key=x.index)))

# # Sort the colors
# df_lands["color"] = df_lands["color"].apply(lambda x: "".join(sorted((c for c in x if c in "CWUBRG"), key="CWUBRG".index)))

# # Update the original DataFrame with the modified df_lands
# df_all_cards.update(df_lands)

In [ ]:
# df_all_cards.to_csv("output_modified.csv")